In [1]:
!pip install ultralytics
!pip install tensorflow==2.14
!pip install tflite_support==0.4.4
!git clone https://github.com/onnx/onnx-tensorflow.git
!pip install onnx
!pip install onnx_tf

fatal: destination path 'onnx-tensorflow' already exists and is not an empty directory.


In [2]:
from ultralytics import YOLO
import os
from IPython.display import display, Image
from IPython import display
from ultralytics import YOLO
import onnx
from onnx_tf.backend import prepare
from google.colab import drive

import tensorflow as tf
import numpy as np
import tensorflow as tf
display.clear_output()


In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Training Phase


In [4]:
!yolo task=detect mode=train model=yolov8m.pt data=/content/drive/MyDrive/datasets/data.yaml epochs=10 imgsz=640

100% 49.7M/49.7M [00:00<00:00, 224MB/s]
Ultralytics YOLOv8.1.9 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/MyDrive/datasets/data.yaml, epochs=10, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, sav

In [5]:
!zip -r /content/drive/MyDrive/trained_final.zip /content/runs

  adding: content/runs/ (stored 0%)
  adding: content/runs/detect/ (stored 0%)
  adding: content/runs/detect/train/ (stored 0%)
  adding: content/runs/detect/train/results.png (deflated 7%)
  adding: content/runs/detect/train/PR_curve.png (deflated 5%)
  adding: content/runs/detect/train/events.out.tfevents.1707053681.a09e5f1c794e.3016.0 (deflated 94%)
  adding: content/runs/detect/train/args.yaml (deflated 52%)
  adding: content/runs/detect/train/R_curve.png (deflated 4%)
  adding: content/runs/detect/train/val_batch2_labels.jpg (deflated 5%)
  adding: content/runs/detect/train/labels_correlogram.jpg (deflated 28%)
  adding: content/runs/detect/train/val_batch0_labels.jpg (deflated 7%)
  adding: content/runs/detect/train/train_batch1.jpg (deflated 11%)
  adding: content/runs/detect/train/train_batch2.jpg (deflated 9%)
  adding: content/runs/detect/train/val_batch1_pred.jpg (deflated 4%)
  adding: content/runs/detect/train/confusion_matrix.png (deflated 17%)
  adding: content/runs/dete

#Exporting As Tflite File



In [41]:
get_ipython().system('yolo task=detect mode=export model=/content/runs/detect/train/weights/last.pt format=onnx')

Ultralytics YOLOv8.1.9 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 218 layers, 25846708 parameters, 0 gradients, 78.7 GFLOPs

PyTorch: starting from '/content/runs/detect/train/weights/last.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 16, 8400) (49.6 MB)

ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: export success ✅ 2.6s, saved as '/content/runs/detect/train/weights/last.onnx' (98.8 MB)

Export complete (9.0s)
Results saved to /content/runs/detect/train/weights
Predict:         yolo predict task=detect model=/content/runs/detect/train/weights/last.onnx imgsz=640  
Validate:        yolo val task=detect model=/content/runs/detect/train/weights/last.onnx imgsz=640 data=/content/drive/MyDrive/datasets/data.yaml  
Visualize:       https://netron.app
💡 Learn more at https://docs.ultralytics.com/modes/export


In [48]:
%cd onnx-tensorflow


/content/onnx-tensorflow


In [49]:
!dir


doc	 LICENSE      onnx_tf		   README.md  setup.py	third_party  Versioning.md
example  MANIFEST.in  ONNX_VERSION_NUMBER  setup.cfg  test	util	     VERSION_NUMBER


In [51]:
#onnx to tfile
onnx_model = onnx.load("/content/runs/detect/train/weights/last.onnx")
tf_rep = prepare(onnx_model)
tf_model_path="/content/drive/MyDrive/TensorFlow"
tf_rep.export_graph(tf_model_path)
model = tf.saved_model.load(tf_model_path)
model.trainable = False

input_tensor = tf.random.uniform([1, 3, 640, 640])
out = model(**{'images': input_tensor})
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS
]
tflite_model = converter.convert()
tflite_model_path="/content/drive/MyDrive/TensorFlow/mymodel.tflite"
# Save the model
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

INFO:absl:Function `__call__` contains input name(s) x, y with unsupported characters which will be renamed to transpose_256_x, add_81_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.
INFO:absl:Writing fingerprint to /content/drive/MyDrive/TensorFlow/fingerprint.pb


In [52]:
#tfile Interface

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path="/content/drive/MyDrive/TensorFlow/mymodel.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# get_tensor() returns a copy of the tensor data
# use tensor() in order to get a pointer to the tensor
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[[     4.9118      21.589      33.483 ...      570.11      591.69      603.88]
  [     8.0384       5.808      6.6816 ...      636.65      637.46      634.74]
  [     9.8283      41.074      61.846 ...      292.66       273.6      245.26]
  ...
  [ 2.4743e-06  7.2097e-07  9.2456e-07 ...  5.7697e-05  4.8772e-05  4.5716e-05]
  [ 4.6251e-07   2.523e-07  2.5394e-07 ...  6.1852e-06  6.8535e-06  1.7493e-05]
  [ 1.7366e-06  9.4187e-07  1.1619e-06 ...  3.8967e-06  5.8467e-06  1.5832e-05]]]
